In [1]:
import numpy as np

# Implementing Black-Scholes Formula in Python

This notebook aims to implement the Black-Scholes options pricing model for a European call option using historical stock price and volatility data. 

**Assumptions** 
The model assumes the following: 
1. The underlying follows a Normal distribution/ Brownian motion
1. There are no arbitrage opportunities in the market
1. The risk-free interest rate is known and constant
1. (There are more)

**Equation**
We have the following parameters in the following equations:
$$
\begin{aligned}
S(t) &= \text{Price of underlying asset at time t} \\ 
V(S, t) &= \text{Price of option given the price of the underlying at time t} \\
K &= \text{Strike price of option} \\
\sigma &= \text{Volatility} \\
r &= \text{Risk-free interest rate} \\
\Phi(d) &= \text{CDF of a normally distributed variable} = \frac{1}{\sqrt{2\pi}} \int^d_{-\infty}e^{\frac{-y^2}{2}}dy
\end{aligned}
$$
The Black Scholes model is a partial differential equation that, given the proper change of variables, can be reduced to the heat equation and solved. The partial differential equation for a European call option is: 
$$
\frac{\delta V}{\delta t} + \frac{1}{2}\sigma^2S^2\frac{\delta^2 V}{\delta S^2} + rS\frac{\delta V}{\delta S}-rV = 0
$$
The full solution for a European call option is: 
$$

$$